For this section, we will solve the problems from HW1 and HW2 using steepest descent without the use of CVX or MOSEK

## Part A: Solving with the initial condition at the origin

### One sensor and three anchors
All of the following code is identical to the setup for HW1 and HW2

In [25]:
import cvxpy as cvx
import numpy as np

In [26]:
# Construct the anchor points
a1 = np.array([1.0, 0.0])
a2 = np.array([-1.0, 0.0])
a3 = np.array([0.0, 2.0])


# Construct the vectors of our triangle
v1 = a1 - a2
v2 = a3 - a2

In [27]:
# Function to generate a random point inside or outside of 
# our convex hull. 
def point_generator(outside=False):
    # Compute a random point within the quadrilateral
    rand_point = a2 + (np.random.random() * v1 + np.random.random() * v2)
    
    # Compute the barycentric coordinates to determine whether the point is inside
    # the triangle, or outside of it
    alpha = ((a2[1] - a3[1])*(rand_point[0] - a3[0]) + (a3[0] - a2[0])* \
            (rand_point[1] - a3[1])) / ((a2[1] - a3[1])*(a1[0] - a3[0]) + \
            (a3[0] - a2[0])*(a1[1] - a3[1]))
    beta = ((a3[1] - a1[1])*(rand_point[0] - a3[0]) + (a1[0] - a3[0])* \
            (rand_point[1] - a3[1])) / ((a2[1] - a3[1])*(a1[0] - a3[0]) + \
            (a3[0] - a2[0])*(a1[1] - a3[1]))
    gamma = 1.0 - alpha - beta

    # If each barycentric coordinate is greater than 0, the point is inside
    if alpha > 0.0 and beta > 0.0 and gamma > 0.0:
        # If we want the point outside
        if outside:
            # Repeat the procedure
            return point_generator(outside)
        else:
            # Return our random point inside the triangle
            return rand_point
    else: # The point is outside
        if outside:
            # Return our random point outside the triangle
            return rand_point
        else:
            # Repeat the procedure
            return point_generator()

In [104]:
# Construct our random point
p = point_generator()

# Compute the Euclidian distances to the anchor points
d1 = np.linalg.norm(p - a1)
d2 = np.linalg.norm(p - a2)
d3 = np.linalg.norm(p - a3)

# Store the anchors, distances and initial starting point at origin
a = [a1, a2, a3]
d  = [d1, d2, d3]
x_0 = np.array([0.,0.])

In [105]:
#Compute the objective function as enumerated in model (1)
def SNL_HW1(a, x, d):
    sum_obj = 0
    for i in range(3):
        sum_obj = sum_obj + ((np.linalg.norm(a[i] - x) ** 2) - d[i] ** 2) ** 2
    
    return sum_obj

#The gradient of the objective function
def dSNL_HW1(a, x, d):
    sum_x = np.zeros(2)
    for i in range(3):
        inner = 2 * (np.linalg.norm(a[i] - x) ** 2 - d[i] ** 2)
        grad = 2 * (-a[i] + x)
        sum_x = sum_x + inner * grad
    
    return sum_x

#Compute the steepest descent method with alpha backtracking
def steepest_descent(op, dop, a, xin, d, niter):
    x = np.copy(xin)
    
    for i in range(0, niter):
        direction = dop(a, x, d)
        alpha = 1.
        while(op(a, x  - alpha * direction, d) > op(a, x, d) \
              0.5 * alpha * np.dot(direction, direction)):
            alpha *= 0.9
        x -= alpha * direction
        
    return x

In [122]:
soln = steepest_descent(SNL_HW1, dSNL_HW1, a, x_0, d, 5)
print "The true point value is", p
print "Solving for the optimal solution, we get: ", soln

The true point value is [-0.0930494   0.95350691]
Solving for the optimal solution, we get:  [-0.09295538  0.95353915]


Therefore, we see that solving for the true value with the starting point at the origin yields the correct solution. Now, let's try the same computation for the problem seen in HW2.

### Two sensors, Three Anchors

In [126]:
# Note that the summations are unrolled for the ease of coding.
# The objective function is for the distances d11, d12, d22, d23, dhat12

# We update the objective function seen with two sensors and three anchors
def SNL_HW2(a, x, d):
    sum_obj = 0
    sum_obj = sum_obj + ((np.linalg.norm(a[0] - x[0]) ** 2) - d[0][0] ** 2) ** 2
    sum_obj = sum_obj + ((np.linalg.norm(a[1] - x[0]) ** 2) - d[0][1] ** 2) ** 2
    sum_obj = sum_obj + ((np.linalg.norm(a[1] - x[1]) ** 2) - d[1][0] ** 2) ** 2
    sum_obj = sum_obj + ((np.linalg.norm(a[2] - x[1]) ** 2) - d[1][1] ** 2) ** 2
    sum_obj = sum_obj + ((np.linalg.norm(x[0] - x[1]) ** 2) - d[2] ** 2) ** 2
    
    return sum_obj

# Here, we perform the gradient of the objective function
# Note that we keep the gradients for the two different
# sensors separate and stored in sum_x[0] and sum_x[1]
# respectively
def dSNL_HW2(a, x, d):
    sum_x = np.zeros((2,2))
    sum_x[0] = sum_x[0] + 4 * \
                (np.linalg.norm(a[0] - x[0]) ** 2 - d[0][0] ** 2) * (-a[0] + x[0])
    sum_x[0] = sum_x[0] + 4 * \
                (np.linalg.norm(a[1] - x[0]) ** 2 - d[0][1] ** 2) * (-a[1] + x[0])
    sum_x[0] = sum_x[0] + 4 * \
                (np.linalg.norm(x[0] - x[1]) ** 2 - d[2] ** 2) * (x[0] - x[1])
    
    sum_x[1] = sum_x[1] + 4 * \
                (np.linalg.norm(a[1] - x[1]) ** 2 - d[1][0] ** 2) * (-a[1] + x[1])
    sum_x[1] = sum_x[1] + 4 * \
                (np.linalg.norm(a[2] - x[1]) ** 2 - d[1][1] ** 2) * (-a[2] + x[1])
    sum_x[1] = sum_x[1] + 4 * \
                (np.linalg.norm(x[0] - x[1]) ** 2 - d[2] ** 2) * (x[1] - x[0])
    
    return sum_x

# Updated steepest descent where we perform it twice for the two
# different sensors
def steepest_descent_2(op, dop, a, xin, d, niter):
    x = np.copy(xin)
    
    for i in range(0, niter):
            direction = dop(a, x, d)
            for j in range(2):
                alpha = 1.
                while(op(a, x[j]  - alpha * direction[j], d) > op(a, x[j], d) \
                      - 0.5 * alpha * np.dot(direction[j], direction[j])):
                    alpha *= 0.9
                x[j] -= alpha * direction[j]
    
    return x

In [118]:
p1 = np.array([-0.4, 0.2])
p2 = np.array([-0.1, 1.7])
x2 = [np.array([0.,0.]), np.array([0.,0.])]

d2 = [[np.linalg.norm(p1 - a1), np.linalg.norm(p1 - a2)], \
      [np.linalg.norm(p2 - a2), np.linalg.norm(p2 - a3)], \
       np.linalg.norm(p1 - p2)]


In [127]:
soln = steepest_descent_2(SNL_HW2, dSNL_HW2, a, x2, d2, 20)
print "The true location of the sensors are: ", p1, p2
print "The solution recovered for the two sensors is: "
print soln[0]
print soln[1]

The true location of the sensors are:  [-0.4  0.2] [-0.1  1.7]
The solution recovered for the two sensors is: 
[ -5.67228040e-01  -1.66533454e-16]
[ 0.30037866  0.86776058]


In contrast to the solution seen with one sensor and three anchors, we do not recover the correct solution. This is because our initial conditions do not allow for convergence to the correct value (unstable equilibrium), we will see this corrected in the next section.

## Part B: Initial condition is at the SOCP/SDP solution

Now, we try to solve the same optimization problems with the initial condition at the SOCP solutions. Note that in HW1 and HW2, we saw that the SOCP/SDP solution were at exact true points with no error, so we replace the initial starting condition at the true points and see if they remain at a fixed point.

### One sensor, three anchors

In [123]:
soln = steepest_descent(SNL_HW1, dSNL_HW1, a, p, d, 5)
print "The true point value is", p
print "Solving for the optimal solution, we get: ", soln

The true point value is [-0.0930494   0.95350691]
Solving for the optimal solution, we get:  [-0.0930494   0.95350691]


It is expected that the one sensor and three anchor problem solution can be optimized to the exact true location since we were able to recover the correct solution starting at the origin.

## Two sensors, three anchors

In [125]:
soln = steepest_descent_2(SNL_HW2, dSNL_HW2, a, [p1, p2], d2, 20)
print "The true location of the sensors are: ", p1, p2
print "The solution recovered for the two sensors is: ", soln[0], soln[1]

The true location of the sensors are:  [-0.4  0.2] [-0.1  1.7]
The solution recovered for the two sensors is:  [-0.4  0.2] [-0.1  1.7]


Interestingly, we see that we indeed recover the correct solution demonstrating that the gradient descent is optimizing the correct objective function. So we see that the solution is extremely sensitive to the initial conditions.